In [1]:
# !pip install bs4
# !pip uninstall BeautifulSoup4 --yes

In [2]:
from bs4 import BeautifulSoup as bs

In [3]:
!pip list

Package                            Version    
---------------------------------- -----------
alabaster                          0.7.12     
anaconda-client                    1.7.2      
anaconda-navigator                 1.9.6      
anaconda-project                   0.8.2      
asn1crypto                         0.24.0     
astroid                            2.1.0      
astropy                            3.1        
atomicwrites                       1.2.1      
attrs                              18.2.0     
Babel                              2.6.0      
backcall                           0.1.0      
backports.os                       0.1.1      
backports.shutil-get-terminal-size 1.0.0      
beautifulsoup4                     4.11.1     
bitarray                           0.8.3      
bkcharts                           0.2        
blaze                              0.11.3     
bleach                             3.0.2      
bokeh                              1.0.2      
boto         

In [4]:
# Dependencies
import os
import sys
import requests
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 as ps
import pandas.io.sql as sqlio
from bs4 import BeautifulSoup as bs

In [5]:
# Read HTML from file. Below url is used to find  the top biofuel stocks. 
#Limited entry to 6 csv files for each ticker due to size limitations of Github upload.
url = 'https://investingnews.com/daily/tech-investing/cleantech-investing/us-biofuel-stocks/'

In [6]:
html = requests.get(url)

In [7]:
soup = bs(html.text,"html.parser")

In [8]:
#Investigation
soup

<!DOCTYPE html>
<!-- Powered by RebelMouse. https://www.rebelmouse.com --><html lang="en"><head prefix="og: http://ogp.me/ns# article: http://ogp.me/ns/article#"><link as="font" crossorigin="" href="https://fonts.gstatic.com/s/opensans/v26/memvYaGs126MiZpBA-UvWbX2vVnXBbObj2OVTS-mu0SC55I.woff2" rel="preload" type="font/woff2"/><meta content="max-image-preview:large" name="robots"/><link as="image" fetchpriority="high" imagesrcset="https://investingnews.com/media-library/tree-stump-from-above.jpg?id=27736632&amp;width=1200&amp;quality=85 3x, https://investingnews.com/media-library/tree-stump-from-above.jpg?id=27736632&amp;width=1200&amp;quality=85 2x, https://investingnews.com/media-library/tree-stump-from-above.jpg?id=27736632&amp;width=500&amp;height=334&amp;quality=85&amp;coordinates=0%2C0%2C0%2C0 1x" rel="preload"/><meta content="summary_large_image" name="twitter:card"/><meta content="@digmedia" name="twitter:site"/><meta content="" name="twitter:site:id"/><meta content="https://inv

In [9]:
# pulling the first title
soup.find("div",class_="rebellt-item stock-data--container col1").find("h3").text.strip()

'1. Aemetis (NASDAQ:AMTX)'

In [10]:
# find the class that our title is in
soup.find_all("div",class_="rebellt-item stock-data--container col1")

[<div class="rebellt-item stock-data--container col1" data-basename="1-aemetis-nasdaq-amtx" data-href="https://investingnews.com/us-biofuel-stocks/1-aemetis-nasdaq-amtx" data-id="4" data-is-image="False" data-post-id="2655226069" data-published-at="1660858494" data-reload-ads="false" data-use-pagination="False" id="rebelltitem4"><h3 data-role="headline">
             
                 1. <a href="https://investingnews.com/company-profiles/aemetis-inc/" target="_blank">Aemetis (NASDAQ:AMTX)</a></h3><script type="application/ld+json">                {"@context":"http://schema.org","@type":"Corporation","name":"Aemetis Inc","url":"https://www.aemetis.com","description":"Aemetis Inc is an international renewable fuel and biochemicals company focused on the production of renewable fuels and chemicals through the acquisition, development and commercialization of technologies that replace traditional petroleum-based products through the conversion of first-generation ethanol and biodiesel pla

In [11]:
# Print all six headlines
tds = soup.find_all("div",class_="rebellt-item stock-data--container col1")
# A blank list to hold the headlines
headlines = []
# Loop over td elements
for td in tds:
    # If td element has an anchor...
    if (td.a):
        print(td.a)
        # And the anchor has non-blank text...
#         if (td.a.text):
            # Append the td to the list
        headlines.append(td.a.text)

<a href="https://investingnews.com/company-profiles/aemetis-inc/" target="_blank">Aemetis (NASDAQ:AMTX)</a>
<a href="https://investingnews.com/company-profiles/alto-ingredients-inc/" target="_blank">Alto Ingredients (NASDAQ:ALTO)</a>
<a href="https://investingnews.com/company-profiles/comstock-inc/" target="_blank">Comstock (NYSE:LODE)</a>
<a href="https://investingnews.com/company-profiles/futurefuel-corp/" target="_blank">FutureFuel (NYSE:FF)</a>
<a href="https://investingnews.com/company-profiles/gevo-inc/" target="_blank">Gevo (NASDAQ:GEVO)</a>
<a href="https://investingnews.com/company-profiles/rex-american-resources-corporation/" target="_blank">REX American Resources (NYSE:REX)</a>


In [12]:
# Print only the headlines that are recived thru WebScrapping
company = []
ticker = []
exchange = []
for x in range(6):
    #print(headlines[x])  
    temp = headlines[x].split(" (")
    #print(temp)
    
    company.append(temp[0])
    tempexchange = temp[1].split(":")
    exchange.append(tempexchange[0])
    ticker.append(tempexchange[1][:-1])
print("company - " ,company)
print("exchange - ",exchange)
print("ticker - ",ticker)


company -  ['Aemetis', 'Alto Ingredients', 'Comstock', 'FutureFuel', 'Gevo', 'REX American Resources']
exchange -  ['NASDAQ', 'NASDAQ', 'NYSE', 'NYSE', 'NASDAQ', 'NYSE']
ticker -  ['AMTX', 'ALTO', 'LODE', 'FF', 'GEVO', 'REX']


In [13]:
#DataFrame is created and will be store eventually to a PostGres Table in Transformed format.
df = pd.DataFrame({'Company' : company, 'Exchange' : exchange , 'Ticker' : ticker})
# print(df.columns[0])
# print(df.iloc[0:1])
df

,Company,Exchange,Ticker
0,Aemetis,NASDAQ,AMTX
1,Alto Ingredients,NASDAQ,ALTO
2,Comstock,NYSE,LODE
3,FutureFuel,NYSE,FF
4,Gevo,NASDAQ,GEVO
5,REX American Resources,NYSE,REX


In [14]:
df_test = pd.DataFrame()
altodf = pd.read_csv('Resources/ALTO.csv')
altodf.head()
x = pd.concat([df_test,altodf])


In [15]:
# Joining the dataframe and CSV files together:

# Read thru each tickets CSV file and merge the entry to above dataframe.
directory = 'Resources'
all_stocks_df = pd.DataFrame()

# loop to find all files in directory
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    
#   loop to find each ticker in ticker list
#     for each in ticker:
    for x,y,z in zip(df['Ticker'],df['Company'],df['Exchange']):
#         print(x,y,z)
       #if ticker in loop ends with ticker name and ".csv"...
        if filename.endswith(x +".csv"): 
           #then read that tickers stock data file
            clean_df = pd.read_csv(os.path.join(directory, filename))
            # print(clean_df)
            # append the ticker name in clean_df and stock info
            # add column and fill in with df fields
            clean_df['Ticker'] = x
            clean_df['Company'] = y
            clean_df['Exchange'] = z
            cleaned_df = pd.concat([all_stocks_df,clean_df])
            all_stocks_df = cleaned_df
            continue
        else:
            continue


In [27]:
# Rearrange columns in dataframe
# print(all_stocks_df)
all_stocks_df = all_stocks_df.loc[:,['Ticker','Company','Exchange','Date', 'Open','High','Low','Close','Adj Close']]
# print(all_stocks_df)

In [17]:
# export dataframe and look over manually to see if data is correct
# IT IS!
os.makedirs('Resources', exist_ok=True)  
all_stocks_df.to_csv('Resources/clean_df_test.csv') 


In [21]:
protocol = 'postgresql'
username = 'postgres'
password = '2022'
host = 'localhost'
port = 5432
database_name = 'ETL'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

# connection_string = "postgres:postgres@localhost:5432/stocks_db"
# engine = create_engine(f'postgresql://{connection_string}')

In [22]:
# Confirm tables
engine.table_names()

[]

In [24]:
all_stocks_df.to_sql(name='stocks', con=engine, if_exists='replace', index=False)